In [1]:
import pickle
with open('train_sentence_transformer.pkl', 'rb') as f:
    train = pickle.load(f)
with open('val_sentence_transformer.pkl', 'rb') as f:
    val = pickle.load(f)
with open('test_sentence_transformer.pkl', 'rb') as f:
    test = pickle.load(f)

In [2]:
import json
with open('test.json', encoding = 'utf8') as json_file:
    data = json.load(json_file)
    
conversations = []
summaries = []
for i in range(0, len(data)):
    if len(data[i]['dialogue'].split('\r\n')) > 1:
        sentences = data[i]['dialogue'].split('\r\n')
    else:
        sentences = data[i]['dialogue'].split('\n')

    conversations.append(sentences)
    summaries.append(data[i]['summary'].strip('\n').replace('\r\nt', ' '))

In [3]:
import numpy as np
length = []
for i in range(0, len(train)):
    length.append(len(train[i]))
for i in range(0, len(val)):
    length.append(len(val[i]))
for i in range(0, len(test)):
    length.append(len(test[i]))
X = []
for i in range(0, len(train)):
    for j in range(0, len(train[i])):
        X.append(np.array(train[i][j]))
for i in range(0, len(val)):
    for j in range(0,len(val[i])):
        X.append(np.array(val[i][j]))
for i in range(0, len(test)):
    for j in range(0, len(test[i])):
        X.append(np.array(test[i][j]))

In [5]:
from hmmlearn import hmm

In [6]:
remodel = hmm.GaussianHMM(n_components=4, n_iter = 50, covariance_type = 'diag', verbose = True, init_params="cm", params="cmts")
remodel.startprob_ = np.array([1, 0.0, 0.0, 0.0])
remodel.transmat_ = np.array([
                 [0.33, 0.34, 0.33, 0],
                 [0.0, 0.33, 0.34, 0.33],
                 [0.0, 0.0, 0.5, 0.5],
                 [0.0, 0.0, 0.0, 1.0]]
                            )

In [7]:
remodel.startprob_

array([1., 0., 0., 0.])

In [8]:
remodel.transmat_

array([[0.33, 0.34, 0.33, 0.  ],
       [0.  , 0.33, 0.34, 0.33],
       [0.  , 0.  , 0.5 , 0.5 ],
       [0.  , 0.  , 0.  , 1.  ]])

In [9]:
remodel.fit(X, length)

         1 -273718.73606499             +nan
         2 -264893.64643455   +8825.08963044
         3 -264603.10500674    +290.54142782
         4 -264553.48144860     +49.62355814
         5 -264548.85978980      +4.62165880
         6 -264474.39117711     +74.46861268
         7 -264467.78838722      +6.60278990
         8 -264467.78837054      +0.00001667


GaussianHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
            covars_weight=1, implementation='log', init_params='cm',
            means_prior=0, means_weight=0, min_covar=0.001, n_components=4,
            n_iter=50, params='cmts', random_state=None, startprob_prior=1.0,
            tol=0.01, transmat_prior=1.0, verbose=True)

In [10]:
remodel.transmat_

array([[0.81951393, 0.14146206, 0.03902402, 0.        ],
       [0.        , 0.71013683, 0.26087553, 0.02898764],
       [0.        , 0.        , 0.90839669, 0.09160331],
       [0.        , 0.        , 0.        , 1.        ]])

In [11]:
remodel.startprob_

array([1., 0., 0., 0.])

In [12]:
def encode_convs(profix):
    sent_label = []
    with open(profix + '_sentence_transformer.pkl', 'rb') as f:
        data = pickle.load(f)
    for i in range(0, len(data)):
        labels = remodel.decode(np.array(data[i]))[1]
        sent_label.append(labels)
    
    with open(profix + '_sent_trans_cons_label.pkl', 'wb') as f:
        pickle.dump(sent_label, f)
    return sent_label

In [21]:
l = encode_convs('train')

In [22]:
l = encode_convs('val')

In [23]:
l = encode_convs('test')